In [1]:
import numpy as np
import pandas as pd
import csv
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
file_path = r"data/real_time_deauth.csv"
df = pd.read_csv(file_path, header=0, usecols=['Time','Protocol','Length','Info'])

Data Cleaning

In [3]:
#drop_protocol = df[df['Protocol'] != '802.11'].index
#df = df.drop(drop_protocol, axis=0)
df['IAT'] = df['Time'].diff()
df['Info'] = (df['Info'].str[:6] == 'Deauth').astype(int)
df = df.drop([0]).reset_index(drop=True).rename(columns={'Info':'Label'})


In [4]:
df = df.drop(['Time','Protocol'], axis=1 )
df = df[['Length','IAT','Label']]

Data Preprocessing

In [5]:
train, valid, test = np.split(df.sample(frac=1),[int(0.6*len(df)), int(0.8*len(df))])

In [6]:
def scale(dataframe):
    X = dataframe[dataframe.columns[:-1]].values
    y = dataframe[dataframe.columns[-1]].values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    data = np.hstack((X, np.reshape(y, (-1,1)))) 

    return data, X, y

In [7]:
train, x_train, y_train = scale(train)
valid, x_valid, y_valid = scale(test)
test, x_test, y_test = scale(test)

SVM

In [8]:
svm_model = SVC(kernel='linear')
svm_model = svm_model.fit(x_train, y_train)

In [9]:
y_pred = svm_model.predict(x_test)

In [ ]:
print(classification_report(y_test, y_pred, digits=4))

In [ ]:
coefficients = svm_model.coef_[0]
intercept = svm_model.intercept_[0]
hyperplane_equation = ' + '.join([f'({coefficients[i]:.3f} * x{i+1})' for i in range(len(coefficients))]) + f' + ({intercept:.3f}) = 0'
print("Hyperplane equation:")
print(hyperplane_equation)

In [12]:
cm = confusion_matrix(y_test, y_pred)
cm_normalized = cm / cm.sum(axis=1, keepdims=True)

In [ ]:
sns.heatmap(cm_normalized, annot=True, fmt="f", cmap="Blues")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()